In [0]:
import numpy as np
import tensorflow  as tf
from tensorflow.contrib.layers import fully_connected
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import librosa
from google.colab import files
from google.colab import drive
import glob
import pickle
from statistics import mean
import keras

Using TensorFlow backend.


In [0]:
drive.mount('/content/Drive/',force_remount =  True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/Drive/


In [0]:
glob.glob('/content/*')

['/content/Drive', '/content/sample_data']

In [0]:
dirpath = "Drive/My Drive/Deep Learning/Project/"

In [0]:
#Loading pickle to colab - Training
sound_lists_pkl_in = open(dirpath+"train_sound_lists.pkl", "rb")
train_sound_lists = pickle.load(sound_lists_pkl_in)

In [0]:
# train_sound_lists

In [0]:
labels = []
for item in train_sound_lists['labels']:
  if item == 'Hindi':
    labels.append(0)
  elif item == 'Bhojpuri':
    labels.append(1)
  else:
    labels.append(2)
    
print(len(labels))
print(labels[:10])
print(train_sound_lists['labels'][:10])

195
[0, 1, 0, 1, 0, 2, 1, 2, 0, 1]
['Hindi', 'Bhojpuri', 'Hindi', 'Bhojpuri', 'Hindi', 'Korean South', 'Bhojpuri', 'Korean South', 'Hindi', 'Bhojpuri']


In [0]:
# train_sound_lists['spec_abs_list'][0].shape
train = []
for i in range(0,195):
  train.append(train_sound_lists['at_list'][i])

In [0]:
train[0].shape

(440111,)

In [0]:

for i in range(len(train)):
  train[i] = librosa.feature.mfcc(y=train[i], n_mfcc = 50)
  train[i] = train[i].T

In [0]:
train[0].shape

(860, 50)

In [0]:
X = tf.placeholder(tf.float32, shape=(None,860,50))
y = tf.placeholder(tf.int32, shape=(None))

In [0]:
x_image_2d = tf.reshape(X, [-1,860,50,1])

In [0]:
#2D Convolutional Layer 1
conv_2d_layer_1 = tf.layers.conv2d(x_image_2d, filters=32, kernel_size=4, strides=(1,1), padding="same")
conv_2d_layer_1 = tf.nn.relu(conv_2d_layer_1)

#2D Maxpooling layer
mp_2d_1 = tf.layers.max_pooling2d(conv_2d_layer_1, pool_size=[2,2], strides=[2,2], padding="same")

#2D Convolutional Layer 2
conv_2d_layer_2 = tf.layers.conv2d(mp_2d_1, filters=64, kernel_size=2, strides=(1,1), padding="same")
conv_2d_layer_2 = tf.nn.relu(conv_2d_layer_2)

#2D Maxpooling layer
mp_2d_2 = tf.layers.max_pooling2d(conv_2d_layer_2, pool_size=[2,2], strides=[2,2], padding="same")

# num_features_2d = mp_2d_2.get_shape()[1:4].num_elements()
# dense_layer_2d = tf.reshape(mp_2d_2, [-1,num_features_2d])

#LSTM
num_features_2d = mp_2d_2.get_shape()[2:4].num_elements()
dense_layer_2d = tf.reshape(mp_2d_2, [-1,mp_2d_2.get_shape()[1],num_features_2d])
#dense_layer_2d = tf.transpose(dense_layer_2d, perm=[0,2,1])

lstm_cell1 = tf.keras.layers.LSTMCell(units=256)

lstm_cell2 = tf.keras.layers.LSTMCell(units=512)

multilayer_lstm = tf.keras.layers.StackedRNNCells([lstm_cell1, lstm_cell2])

rnn_outputs, rnn_states = tf.nn.dynamic_rnn(multilayer_lstm, dense_layer_2d, dtype=tf.float32)
print(rnn_outputs)
rnn_outputs = tf.reshape(rnn_outputs, [-1,rnn_outputs.shape[1]*rnn_outputs.shape[2]])
# rnn_outputs = rnn_outputs[:,-1,:]
# rnn_outputs.shape
#Fully Connected Layer
dl_2d_1 = fully_connected(rnn_outputs, 256)

#Dropout Layer 1
dr_2d_1 = tf.layers.dropout(dl_2d_1, rate=0.7)

#Output Layer
reg_op_2d = fully_connected(dr_2d_1, 3, activation_fn=None)
print(reg_op_2d.shape)


Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Tensor("rnn/transpose_1:0", shape=(?, 215, 512), dtype=float32)
Instructions for updating:
Use keras.layers.dropout instead.
(?, 3)


In [0]:
#Mean Square Error(MSE) - Defining the error calculated using output of the NN and the given output
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=reg_op_2d)
loss_2d = tf.reduce_mean(xentropy, name="loss")

correct = tf.nn.in_top_k(reg_op_2d, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32)) 



In [0]:
#Using Adam Optimizer for minimizing the loss
lr = 0.0002
optimizer2 = tf.train.AdamOptimizer(lr).minimize(loss_2d)

In [0]:
init = tf.global_variables_initializer()
#saver = tf.train.Saver()

In [0]:
#Training the NN
n_epochs =50
batch_size = 29
acc_2d = []
tx = train[:145]
cvx = train[145:]
ty = labels[:145]
cvy = labels[145:]
with tf.Session() as sess:
  init.run()
  for epoch in range(n_epochs):
    for iteration in range(len(tx) // batch_size):
      
      X_batch = tx[iteration*batch_size:(iteration+1)*batch_size]
      y_batch = ty[iteration*batch_size:(iteration+1)*batch_size]
      sess.run(optimizer2, feed_dict = {X: X_batch, y: y_batch})
    acc_train = accuracy.eval(feed_dict={X: tx, y: ty})
    acc_2d.append(acc_train*100)
    #if epoch % 100 == 0:
    print(epoch, "Train accuracy:", acc_train)
    #, "Test accuracy:", acc_test
  
  acc_test = accuracy.eval(feed_dict={X: cvx, y: cvy})
  print(acc_test)
  pred = tf.arg_max(tf.nn.softmax(reg_op_2d.eval(feed_dict={X: cvx})),1)
  cm = sess.run(tf.math.confusion_matrix(cvy, pred))
  
  

0 Train accuracy: 0.44827586
1 Train accuracy: 0.9310345
2 Train accuracy: 0.8344827
3 Train accuracy: 0.9724138
4 Train accuracy: 1.0
5 Train accuracy: 1.0
6 Train accuracy: 1.0
7 Train accuracy: 1.0
8 Train accuracy: 1.0
9 Train accuracy: 1.0
10 Train accuracy: 1.0
11 Train accuracy: 1.0
12 Train accuracy: 1.0
13 Train accuracy: 1.0
14 Train accuracy: 1.0
15 Train accuracy: 1.0
16 Train accuracy: 1.0
17 Train accuracy: 1.0
18 Train accuracy: 1.0
19 Train accuracy: 1.0
20 Train accuracy: 1.0
21 Train accuracy: 1.0
22 Train accuracy: 1.0
23 Train accuracy: 1.0
24 Train accuracy: 1.0
25 Train accuracy: 1.0
26 Train accuracy: 1.0
27 Train accuracy: 1.0
28 Train accuracy: 1.0
29 Train accuracy: 1.0
30 Train accuracy: 1.0
31 Train accuracy: 1.0
32 Train accuracy: 1.0
33 Train accuracy: 1.0
34 Train accuracy: 1.0
35 Train accuracy: 1.0
36 Train accuracy: 1.0
37 Train accuracy: 1.0
38 Train accuracy: 1.0
39 Train accuracy: 1.0
40 Train accuracy: 1.0
41 Train accuracy: 1.0
42 Train accuracy: 

In [0]:
cm

array([[19,  4,  0],
       [ 2, 11,  0],
       [ 0,  0, 14]], dtype=int32)

### Using Keras

In [0]:
    
from keras.layers.core import Dense, Permute, Reshape, Flatten 
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.layers import Dropout, TimeDistributed
from keras import layers
from keras.models import Sequential
from keras.regularizers import l2
from keras.utils import to_categorical

In [0]:
model = Sequential()

model.add(Convolution2D(32, kernel_size=(4,4), strides=(1,1), activation="relu", padding="same", input_shape=(860,50,1)))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))


model.add(Convolution2D(64, kernel_size=(2,2), strides=(1,1), activation="relu", padding="same"))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))

print(model.layers[-1].output_shape)

#model.add(Permute((2,1,3)))

print(model.layers[-1].output_shape)

bs, x, y, c = model.layers[-1].output_shape

model.add(Reshape([x,y*c]))

print(model.layers[-1].output_shape)

model.add(LSTM(256, activation="tanh", return_sequences=True))

model.add(LSTM(512, activation="tanh", return_sequences=True))

model.add(Flatten())

model.add(Dense(256, activation="relu"))

model.add(Dropout(rate=0.7))

model.add(Dense(3, activation="softmax"))

(None, 215, 13, 64)
(None, 215, 13, 64)
(None, 215, 832)


In [0]:
tx = np.array(train[:145])
cvx = np.array(train[145:])
ty = np.array(labels[:145])
cvy = np.array(labels[145:])

In [0]:
ty_onehot = to_categorical(ty)
cvy_onehot = to_categorical(cvy)

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.adam(lr=0.0002),
             metrics=['accuracy'])

In [0]:
model.fit(tx.reshape(-1,860,50,1), ty_onehot,
          batch_size=29,
          epochs=70,
          verbose=1,
          validation_data=(cvx.reshape(-1,860,50,1),cvy_onehot)
         )
          

Train on 145 samples, validate on 50 samples
Epoch 1/70
145/145 [==============================] - 9s 64ms/step - loss: 0.0030 - acc: 1.0000 - val_loss: 0.4829 - val_acc: 0.9600
Epoch 2/70
145/145 [==============================] - 6s 40ms/step - loss: 2.7392e-06 - acc: 1.0000 - val_loss: 0.5851 - val_acc: 0.9600
Epoch 3/70
145/145 [==============================] - 6s 41ms/step - loss: 5.9486e-05 - acc: 1.0000 - val_loss: 0.6447 - val_acc: 0.9600
Epoch 4/70
145/145 [==============================] - 6s 41ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.6447 - val_acc: 0.9600
Epoch 5/70
145/145 [==============================] - 6s 40ms/step - loss: 1.3379e-05 - acc: 1.0000 - val_loss: 0.6038 - val_acc: 0.9600
Epoch 6/70
145/145 [==============================] - 6s 40ms/step - loss: 6.2717e-05 - acc: 1.0000 - val_loss: 0.6447 - val_acc: 0.9600
Epoch 7/70
145/145 [==============================] - 6s 41ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.6447 - val_acc: 0.9600


In [0]:
  from keras import backend as K
  from sklearn.metrics import confusion_matrix

In [0]:
pb = model.predict(cvx.reshape(-1,860,50,1))
y_pred=np.argmax(pb,1)
confusion_matrix(cvy, y_pred)

array([[22,  1,  0],
       [ 1, 12,  0],
       [ 0,  0, 14]])

In [0]:
inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
#functors = [K.function([inp, K.learning_phase()], [out]) for out in outputs]    # evaluation functions

In [0]:
outputs

[<tf.Tensor 'conv2d_7_1/Relu:0' shape=(?, 860, 50, 32) dtype=float32>,
 <tf.Tensor 'batch_normalization_7/cond/Merge:0' shape=(?, 860, 50, 32) dtype=float32>,
 <tf.Tensor 'max_pooling2d_7_1/MaxPool:0' shape=(?, 430, 25, 32) dtype=float32>,
 <tf.Tensor 'conv2d_8/Relu:0' shape=(?, 430, 25, 64) dtype=float32>,
 <tf.Tensor 'batch_normalization_8/cond/Merge:0' shape=(?, 430, 25, 64) dtype=float32>,
 <tf.Tensor 'max_pooling2d_8/MaxPool:0' shape=(?, 215, 13, 64) dtype=float32>,
 <tf.Tensor 'permute_4/transpose:0' shape=(?, 13, 215, 64) dtype=float32>,
 <tf.Tensor 'reshape_4_1/Reshape:0' shape=(?, 13, 13760) dtype=float32>,
 <tf.Tensor 'lstm_6/transpose_1:0' shape=(?, ?, 256) dtype=float32>,
 <tf.Tensor 'lstm_7/TensorArrayReadV3:0' shape=(?, 512) dtype=float32>,
 <tf.Tensor 'dense_5/Relu:0' shape=(?, 512) dtype=float32>,
 <tf.Tensor 'dropout_3_1/cond/Merge:0' shape=(?, 512) dtype=float32>,
 <tf.Tensor 'dense_6/Softmax:0' shape=(?, 3) dtype=float32>]

In [0]:
model.layers[1].output